In [134]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

import pandas as pd
import numpy as np

In [135]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class'] = iris.target 
print(df['class'].unique()) #CLass are [0,1,2] 3 classes 0 : Versicolor, 1 : Setosa, 2 : Virginica 
print("\n",df.info()) #sample is 150 entries, feature 4 data, and target 1 data.
print("\n",df.tail())

[0 1 2]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   class              150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB

 None

      sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     c

In [145]:
#Allow duplication of bagging 10 times to pull out data and make 20 samples. 
#A total of 200 data are created and data that have never been pulled out are stored in Out of data
def Bagging(dataset):
    bagging_df = pd.DataFrame(columns=dataset.columns)
    test_df = pd.DataFrame(columns=dataset.columns)
    Out_of_Data = set(range(dataset.shape[0])) #Initial 200 sample(0~199), not duplication 

    training_index = pd.DataFrame() #save bootstraped index.

    #generate 10 bagging dataset and each with 20 records.
    for i in range(10):
        data_index = list(range(150)) 
        random_data_index = np.random.choice(data_index, 20, replace=True) #restoration extraction
        training_index[f'{i}th'] = random_data_index
        bagging_sample = dataset.iloc[random_data_index, :]
        bagging_df = pd.concat([bagging_df, bagging_sample], axis=0) #Stored in bagging_df via pulled data indices
        Out_of_Data -= set(random_data_index) #remove the sampled data index
    
    Out_of_Data = list(Out_of_Data)
    test_df = dataset.iloc[Out_of_Data, :]
    print("\n Not sampled data index : ",Out_of_Data, "\n Not sampled data length is : ",len(Out_of_Data))
    print("\nThe bootstrap training dataset index\n", training_index)
    return bagging_df, test_df

In [146]:

#How to create #testdataset: Find the index as an intersection that has never been selected in the process of creating 10 bootstraps.
bagging_df, test_df = Bagging(df)

X_train = bagging_df.iloc[:,0:4].values
y_train = bagging_df.iloc[:,-1].values

X_test = test_df.iloc[:,0:4].values
y_test = test_df.iloc[:,-1].values

y_train = y_train.astype(int)



 Not sampled data index :  [14, 15, 30, 33, 36, 42, 53, 58, 62, 63, 64, 71, 74, 75, 82, 83, 89, 98, 101, 103, 114, 118, 119, 123, 126, 127, 131, 139, 141, 142] 
 Not sampled data length is :  30

The bootstrap training dataset index
     0th  1th  2th  3th  4th  5th  6th  7th  8th  9th
0    38  132  124   40   72   97   18  144  110    2
1    54  144   68   41   34   90   39   26  104    4
2   129   59   29  110    5   47  145   24  145   77
3   116  109  104   16   65  117   76    0  135  147
4    19   67   37   68  137  100   46    2  140   95
5    16  111   96   78   49   60   76   79   32   67
6    59   87   29   73  110   67  128   10  134   61
7   125   26   12   88   61   54   73   61   17    3
8    69  129   60   43   66  140   24  100   20   28
9    91   84  102  125   48   72   38   94   85   57
10   51   23  121   95  140    8    9  113   24  115
11  143   28    1   22   50  111   90  100   93    1
12   32  144    7   38   27    7   78  124   55  111
13   86   40   41   81 

C:\Users\ksmin\AppData\Local\Temp\ipykernel_8784\2636670353.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bagging_df = pd.concat([bagging_df, bagging_sample], axis=0) #Stored in bagging_df via pulled data indices


In [140]:
print(X_train)

[[5.4 3.  4.5 1.5]
 [6.3 2.3 4.4 1.3]
 [5.7 2.6 3.5 1. ]
 [6.4 3.1 5.5 1.8]
 [7.7 2.8 6.7 2. ]
 [5.1 3.5 1.4 0.2]
 [4.5 2.3 1.3 0.3]
 [5.5 2.6 4.4 1.2]
 [5.7 4.4 1.5 0.4]
 [7.4 2.8 6.1 1.9]
 [5.  3.5 1.6 0.6]
 [4.6 3.1 1.5 0.2]
 [7.4 2.8 6.1 1.9]
 [5.6 2.9 3.6 1.3]
 [5.4 3.4 1.7 0.2]
 [5.6 3.  4.5 1.5]
 [5.7 2.8 4.1 1.3]
 [7.2 3.2 6.  1.8]
 [5.2 3.5 1.5 0.2]
 [6.  2.2 5.  1.5]
 [5.8 2.6 4.  1.2]
 [5.1 3.8 1.6 0.2]
 [6.8 3.2 5.9 2.3]
 [4.9 3.1 1.5 0.1]
 [6.5 3.  5.2 2. ]
 [5.5 4.2 1.4 0.2]
 [5.5 3.5 1.3 0.2]
 [6.9 3.1 5.4 2.1]
 [5.7 2.9 4.2 1.3]
 [5.9 3.  4.2 1.5]
 [5.2 3.5 1.5 0.2]
 [4.7 3.2 1.3 0.2]
 [6.7 2.5 5.8 1.8]
 [7.7 2.6 6.9 2.3]
 [5.8 2.7 3.9 1.2]
 [6.4 3.1 5.5 1.8]
 [5.8 2.8 5.1 2.4]
 [5.4 3.  4.5 1.5]
 [5.2 3.4 1.4 0.2]
 [5.8 4.  1.2 0.2]
 [6.4 3.2 4.5 1.5]
 [5.5 2.6 4.4 1.2]
 [4.8 3.4 1.6 0.2]
 [5.6 2.7 4.2 1.3]
 [6.8 2.8 4.8 1.4]
 [6.5 3.2 5.1 2. ]
 [5.8 2.8 5.1 2.4]
 [6.1 3.  4.9 1.8]
 [6.7 3.1 4.7 1.5]
 [6.  2.9 4.5 1.5]
 [6.6 2.9 4.6 1.3]
 [4.9 3.1 1.5 0.2]
 [5.1 2.5 3.

In [141]:
print(y_train, "\n y_train length is : ", len(y_train))
print("\n", y_test, "\n y_test length is : ", len(y_test))

[1 1 1 2 2 0 0 1 0 2 0 0 2 1 0 1 1 2 0 2 1 0 2 0 2 0 0 2 1 1 0 0 2 2 1 2 2
 1 0 0 1 1 0 1 1 2 2 2 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 2 1 2 0 0 1 2 0 1 2 0
 2 1 2 0 1 2 1 1 2 0 1 2 2 2 0 2 1 0 2 2 1 0 1 0 0 1 1 1 2 2 2 0 0 2 2 2 2
 2 1 2 2 1 2 1 0 0 0 2 1 1 0 0 0 1 1 2 1 2 0 2 0 2 2 1 0 2 1 2 1 0 2 0 2 0
 1 0 1 1 2 0 1 0 2 0 0 2 2 1 0 1 1 1 1 0 2 0 1 0 0 1 1 2 2 0 2 2 1 0 1 0 1
 2 2 2 0 2 0 1 2 0 1 1 0 1 1 0] 
 y_train length is :  200

 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] 
 y_test length is :  32


In [142]:
def bagging_CF(X_train,X_test,y_train,y_test):
    bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=10,
    bootstrap=True,
    random_state=42
    )
    bagging_clf.fit(X_train,y_train)

    #initialize each base learners prediction
    base_learner_predictions = np.zeros((X_test.shape[0],10))

    for i, estimator in enumerate(bagging_clf.estimators_):
        base_learner_predictions[:,i] = estimator.predict(X_test)

    y_pred = bagging_clf.predict(X_test)

    print(accuracy_score(y_test, y_pred))

    predictions_df = pd.DataFrame(base_learner_predictions, columns=[f'{i+1}' for i in range(10)])

    predictions_df['Ensemble'] = y_pred
    predictions_df["Reality"] = y_test

    confusion_matrix = Confuse_Matrix(predictions_df['Reality'],predictions_df['Ensemble'])
    return predictions_df,confusion_matrix

def Confuse_Matrix(reality, ensemble):
    confusion_matrix = pd.crosstab(reality,ensemble,rownames=['Actual'], colnames=['Predicted'])

    return confusion_matrix

In [143]:
predictions_df,confusion_matrix = bagging_CF(X_train,X_test,y_train,y_test)

print(predictions_df)

0.90625
      1    2    3    4    5    6    7    8    9   10  Ensemble  Reality
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         0        0
11  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0         1        1
12  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0    

In [144]:
print(confusion_matrix)

Predicted   0  1   2
Actual              
0          11  0   0
1           0  4   1
2           0  2  14
